In [1]:
!pip install python-chess
!pip install psutil
!pip install google-cloud-storage

import numpy as np
import pandas as pd
import glob
import os
import chess


In [2]:
import psutil

# Get system memory information
memory = psutil.virtual_memory()
print(f"Total RAM: {memory.total / (1024 ** 3):.2f} GB")
print(f"Used RAM: {memory.used / (1024 ** 3):.2f} GB")
print(f"Free RAM: {memory.free / (1024 ** 3):.2f} GB")


Total RAM: 51.00 GB
Used RAM: 1.20 GB
Free RAM: 47.86 GB


In [3]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gleaming-modem-400906-3780658b0f47.json"
from google.cloud import storage

def download_from_bucket(blob_name, path_to_save_to, bucket_name):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(path_to_save_to)
    print(f"File {blob_name} downloaded to {path_to_save_to}.")

# Use the function to download your file:
download_from_bucket("concatenated_array2.npy", "/content/concatenated_array2.npy", "chess_data_all")
download_from_bucket("concatenated_array1.npy", "/content/concatenated_array1.npy", "chess_data_all")


File concatenated_array2.npy downloaded to /content/concatenated_array2.npy.
File concatenated_array1.npy downloaded to /content/concatenated_array1.npy.


In [2]:


bitboard_dataset=np.load("/content/concatenated_array1.npy")
labels=np.load("/content/concatenated_array2.npy")

In [5]:
!nvidia-smi


Thu Oct  5 16:07:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
print(bitboard_dataset[20])

[[[  0.      0.      0.    ...   0.      0.      0.   ]
  [ 11.     11.      0.    ...   0.     11.     11.   ]
  [  1.      1.      0.    ...   0.      1.      1.   ]
  ...
  [  0.      0.      0.    ...   1.      0.      0.   ]
  [  0.      0.      0.    ...   0.      0.      0.   ]
  [  0.      0.      0.    ...   0.      0.      0.   ]]

 [[  0.      0.      0.    ...   0.      0.      0.   ]
  [  0.      0.      0.    ...   0.      0.      0.   ]
  [  0.      0.      0.    ...   0.      0.      0.   ]
  ...
  [ -1.     -0.125   0.    ... -11.     -1.     -1.   ]
  [-11.    -11.      0.    ...   0.    -11.    -11.   ]
  [  0.      0.      0.    ...   0.      0.      0.   ]]

 [[  0.      0.      4.    ...   0.      4.      0.   ]
  [  0.      0.      0.    ...   0.      0.      0.5  ]
  [  0.      0.      4.    ...  44.      4.      0.   ]
  ...
  [  0.      0.      0.    ...   0.      0.      0.   ]
  [  0.      0.      0.    ...   0.      0.      0.   ]
  [  0.      0.      0.   

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Concatenate
from tensorflow.keras.activations import relu
import tensorflow as tf
from tensorflow.keras.layers import DepthwiseConv2D


# ... Assuming bitboard_dataset and labels are defined ...

bitboard_dataset, labels = shuffle(bitboard_dataset, labels)
X_train, X_test, y_train, y_test = train_test_split(bitboard_dataset, labels, test_size=0.2, random_state=42)


# Input layer
input_layer_2d = layers.Input(shape=(16,8, 8))  # for 2D convolutions
input_layer_3d = layers.Reshape((16, 8, 8, 1))(input_layer_2d)  # for 3D convolutions

# Branch 1: Depthwise Convolution
branch1 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(input_layer_2d)
branch1 = Conv2D(filters=15, kernel_size=(8, 8), activation=relu)(branch1)
branch1 = Flatten()(branch1)  # Flatten branch1
# Branch 2: Conv3D
branch2 = Conv3D(filters=16, kernel_size=(5, 5, 5), padding='valid', activation=relu)(input_layer_3d)
branch2 = Conv3D(filters=8, kernel_size=(3, 3, 3), padding='valid', activation=relu)(branch2)
branch2 = Flatten()(branch2)  # Flatten branch2

# Branch 3: Conv3 (Only to the 15th channel)
branch3_input = tf.gather(input_layer_2d, [13,14], axis=3)  # Extract the 15th channel
branch3 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(branch3_input)
branch3 = Conv2D(filters=10, kernel_size=(8,8), activation=relu)(branch3)
branch3 = Conv2D(filters=2, kernel_size=(1, 1), activation=relu)(branch3)
branch3 = Flatten()(branch3)  # Flatten branch3

concat = Concatenate()([branch1, branch2, branch3])

# Dense layers
dense1 = Dense(64, activation='relu')(concat)
dense2 = Dense(32, activation='relu')(dense1)
dense3 = Dense(16, activation='relu')(dense2)
output_layer = Dense(1)(dense3)  # No activation function for regression

# Create the model
model = Model(inputs=input_layer_2d, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mape'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=1000, validation_split=0.1)

# Evaluate the model
loss, mae, mape = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)